# A27 - Fundamentals and Design of Blockchain-based Systems

In this course you will be designing your own blockchain using the IPv8 peer to peer networking library. The project template provided to you, supplies you with all the tools necessary to perform the lab. The project is structured as follows:

```
asci-a27-blockhain/
|-- bami/                 // contains the IPv8 simulator
|-- |-- ... 
|   
|
|-- simulation/
|   |-- common/           // contains building block for your solution
|   |   |-- ...
|   |   |-- ...
|   |   
|   |-- example/
|   |-- |-- example.py    // contains a simulation example.
|

```

    ## PingPong Community

IPv8 relies on network overlays: virtual networks that are built on top of existing physical networks. The PingPong simulation makes use of such an overlay. Next, we'll explain how this code works.

In [3]:
import os
from asyncio import ensure_future, get_event_loop

from ipv8.community import Community
from ipv8.configuration import ConfigBuilder
from ipv8.lazy_community import lazy_wrapper
from ipv8.messaging.lazy_payload import VariablePayload, vp_compile

from simulation.common.settings import SimulationSettings
from simulation.common.simulation import SimulatedCommunityMixin, BamiSimulation
from simulation.common.utils import time_mark, connected_topology

After the required imports, we define our network overlay `PingPongCommunity` — which IPv8 refers to as a Community. We extend from the IPv8 `Community` class and define our message types and handlers.

In [4]:
@vp_compile
class PingMessage(VariablePayload):
    msg_id = 1

@vp_compile
class PongMessage(VariablePayload):
    msg_id = 2

    
class PingPongCommunity(Community):
    """
    This basic community sends ping messages to other known peers every two seconds.
    """
    community_id = os.urandom(20)

    def __init__(self, my_peer, endpoint, network):
        super().__init__(my_peer, endpoint, network)
        self.add_message_handler(1, self.on_ping_message)
        self.add_message_handler(2, self.on_pong_message)

    def started(self):
        self.register_task("send_ping", self.send_ping, interval=2.0, delay=0)

    def send_ping(self):
        self.logger.info("🔥 <t=%.1f> peer %s sending ping", get_event_loop().time(), self.my_peer.address)
        for peer in self.network.verified_peers:
            self.ez_send(peer, PingMessage())

    @lazy_wrapper(PingMessage)
    def on_ping_message(self, peer, payload):
        self.logger.info("🔥 <t=%.1f> peer %s received ping", get_event_loop().time(), self.my_peer.address)
        self.logger.info("🧊 <t=%.1f> peer %s sending pong", get_event_loop().time(), self.my_peer.address)
        self.ez_send(peer, PongMessage())

    @lazy_wrapper(PongMessage)
    def on_pong_message(self, peer, payload):
        self.logger.info("🧊 <t=%.1f> peer %s received pong", get_event_loop().time(), self.my_peer.address)
        

Next, we setup the BAMI Simulator and add our `PingPong` overlay to the simulator IPv8 nodes.

In [5]:
class BasicPingPongSimulation(BamiSimulation):
    
    def get_ipv8_builder(self, peer_id: int) -> ConfigBuilder:
        builder = super().get_ipv8_builder(peer_id)
        builder.add_overlay("PingPongCommunity", "my peer", [], [], {}, [('started',)])
        return builder

We also use the a mixin to handle delays.

In [6]:
class SimulatedPingPong(SimulatedCommunityMixin, PingPongCommunity):
    send_ping = time_mark(PingPongCommunity.send_ping)
    on_ping_message = time_mark(PingPongCommunity.on_ping_message)


Finally, we define the parameters for our simulation and run it for 10 seconds.

In [7]:
# We use a discrete event loop to enable quick simulations.
if __name__ == "__main__":
    settings = SimulationSettings()
    settings.peers = 6
    settings.duration = 10
    settings.indefinite = True
    settings.topology = connected_topology(settings.peers)
    settings.community_map = {'PingPongCommunity': SimulatedPingPong}

    simulation = BasicPingPongSimulation(settings)
    ensure_future(simulation.run())
    simulation.loop.run_forever()

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<BamiSimulation.run() done, defined at /Users/rowdy/Documents/repos/phd-research/asci-a27-blockchain/simulation/common/simulation.py:178> exception=RuntimeError("Task <Task pending name='Task-4' coro=<BamiSimulation.run() running at /Users/rowdy/Documents/repos/phd-research/asci-a27-blockchain/simulation/common/simulation.py:181>> got Future <_GatheringFuture pending> attached to a different loop")>
Traceback (most recent call last):
  File "/Users/rowdy/Documents/repos/phd-research/asci-a27-blockchain/simulation/common/simulation.py", line 181, in run
    await self.start_ipv8_nodes()
  File "/Users/rowdy/Documents/repos/phd-research/asci-a27-blockchain/simulation/common/simulation.py", line 93, in start_ipv8_nodes
    await instance.start()
  File "/Users/rowdy/Library/Caches/pypoetry/virtualenvs/bami--y7Kd02X-py3.8/lib/python3.8/site-packages/ipv8_service.py", line 144, in start
    await gath

## Lab day 1: the Basics

For the first lab day, we expect you to implement the preliminaries towards a function blockchain. For this, you will be using the IPv8 simulator. We expect you to implement the following:

- A network with full visibility.
- Two different types of nodes: validators and light nodes.
- Balanced based cryptocurrency transactions. Clients should be able to send and receive tokens. This will be done by sending a transaction to a validator node. The validator node will then broadcast the transaction to all other nodes. The outcome of this transaction should be added to a local storage, storing the updated balances from the corresponding parties.
- As an outcome, we expect you to print the contents of the database in order to verify the correctness of the transactions.

## Lab day 2: Consensus

For this second day, we expect you to implement a consensus mechanism for your preliminary blockchain. For this, you will be implementing a Proof of Stake (PoS) consensus mechanism. We expect you to implement the following:
- Blocks
- A mempool
- Mining
- A PoS consensus mechanism using weights based on the amount of tokens a validator has.

As an outcome, play with different stake weights and see how this affects the consensus mechanism. For example, what happens if a validator has 10x more tokens than the other validators? What happens if a validator has 10x less tokens than the other validators?

## Lab day 3: Payment channels

For this third day, we expect you to implement a payment channel mechanism for your preliminary blockchain. We expect you to implement the following:
- Opening a payment channel.
- Making payments.
- Closing a payment channel.

As an outcome, we ask you to play around with different network topologies and report on the differences you observe. For example, what happens if you have a fully connected network? Or a BA graph? What happens if you have a network with a single bottleneck? What happens if you have a network with multiple bottlenecks?

## Lab day 4: Blue vs. Red

In this fourth day, we expect you to implement a malicious node that can attack the blockchain. We ask you to divide into two groups: a blue team and a red team. The blue teams gets to decide amongst themselves what implementation they will pick as their favourite. You will provide the red team with this implementation.

The red team will then implement a malicious node that can attack the blockchain. The red team is free in their approach of attacks, for instance:

- Double spending
- Sybil attacks
- Eclipse attacks
- Byzantine nodes

The end goal is to steal tokens from the blue team. The blue team will then try to defend their blockchain against the red team. The blue team is free in their approach of defending their blockchain. They should critically assess and try to find weaknesses in the implementation. However, it is most important to not change the message format of the blockchain as this would sabotage the red team.

## Lab day 5: Raising the stakes

In this final lab day, we continue with the feud. We ask you to finalize your attacks or defenses. After doing so, you will make the switch to actual IPv8 nodes. You will then run your attacks or defenses on the IPv8 network. We will have a live battle to see who will win the war.